In [1]:
import pandas as pd
import numpy as np

In [49]:
data = pd.read_csv("/home/florian/bachelorarbeit/code/Cross-Silo-FL/datasets/horizontal/covid/owid-covid-data.csv")

In [3]:
data.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


In [50]:

import sys
sys.path.insert(1, '../scripts')
import helper


In [54]:
x_train, y_train = helper.get_samples("covid", 5, ["total_cases", "new_cases"], "new_cases", "", max_samples=1000)
print(y_train[0:2])

[0.0, 0.0]


*Tests to load weather data start here*

In [2]:
import tensorflow as tf

2023-01-03 18:59:26.252891: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-03 18:59:26.695633: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-03 18:59:26.695667: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-03 18:59:27.812762: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
path = "/home/florian/bachelorarbeit/code/Cross-Silo-FL/datasets/vertical/weather/muenchen.csv"

In [4]:
data = pd.read_csv(path, names=["time", "temp", "dwpt", "rhum", "prcp", "snow", "wdir", "wspd", "wpgt", "pres", "tsun", "coco"])
data.head()

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
0,1879-01-01,NaN,4.6,9.8,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1879-01-02,NaN,0.7,8.5,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1879-01-03,NaN,-0.9,8.6,0.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1879-01-04,NaN,0.4,8.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1879-01-05,NaN,-4.7,0.0,5.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
exog_columns = ["temp", "pres", "tsun"]
edog_column = "temp"

In [9]:
tf_dataset = tf.data.Dataset.from_tensor_slices(dict(data))

In [55]:
exog = None
edog = None
x_train = []
y_train = []
for window in tf_dataset.batch(11, drop_remainder=True).take(10000):
    new_df = pd.DataFrame(window)
    exog = new_df[exog_columns].iloc[:10]
    edog = new_df["time"].iloc[-1]

    if not exog.isnull().values.any() or not edog: #check for NaNs
        exog = exog.to_numpy().flatten()


        x_train.append(exog)
        y_train.append(edog)
    #     exog = None
    #     edog = None

print(y_train[0:2])

[b'1985-01-15', b'1985-01-26']
